In [ ]:
import importlib
import multiprocessing as mp
import pandas as pd
import numpy as np
import requests
import datetime
import time
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root = '/content/drive/MyDrive/CIS700/'
df = pd.read_csv(root + 'trip_data_1.csv',usecols=['medallion', ' pickup_datetime', ' dropoff_datetime',
       ' passenger_count', ' trip_time_in_secs', ' trip_distance',
       ' pickup_longitude', ' pickup_latitude', ' dropoff_longitude',
       ' dropoff_latitude'])

In [ ]:
df[' pickup_datetime'] = pd.to_datetime(df[' pickup_datetime'])
df[' dropoff_datetime'] = pd.to_datetime(df[' dropoff_datetime'])


### Create Sharability Network

In [ ]:
class Node:
  def __init__(self, origin, destination, start_time , end_time, id):
    self.id = id
    self.origin = origin
    self.dest = destination
    self.st = start_time
    self.at = end_time
  

In [ ]:
df.iloc[0], df.iloc[1]

(medallion                      2010000001
  pickup_datetime      2010-01-01 00:00:00
  dropoff_datetime     2010-01-01 00:34:00
  passenger_count                        1
  trip_time_in_secs                     34
  trip_distance                      14.05
  pickup_longitude                -73.9484
  pickup_latitude                  40.7246
  dropoff_longitude               -73.9261
  dropoff_latitude                 40.8648
 Name: 0, dtype: object, medallion                      2010000002
  pickup_datetime      2010-01-01 00:00:00
  dropoff_datetime     2010-01-01 00:33:00
  passenger_count                        1
  trip_time_in_secs                     33
  trip_distance                       9.65
  pickup_longitude                -73.9974
  pickup_latitude                  40.7362
  dropoff_longitude               -73.9978
  dropoff_latitude                 40.7362
 Name: 1, dtype: object)

In [ ]:
df = df_sample
o1 = (df.iloc[0][' pickup_longitude'],df.iloc[0][' pickup_latitude'])
d1 = (df.iloc[0][' dropoff_longitude'],df.iloc[0][' dropoff_latitude'])
s1 = df.iloc[0][' pickup_datetime']
t1 = df.iloc[0][' dropoff_datetime']

n1 = Node(o1, d1, s1, t1,1)

o2 = (df.iloc[1][' pickup_longitude'],df.iloc[1][' pickup_latitude'])
d2 = (df.iloc[1][' dropoff_longitude'],df.iloc[1][' dropoff_latitude'])
s2 = df.iloc[1][' pickup_datetime']
t2 = df.iloc[1][' dropoff_datetime']

n2 = Node(o2, d2, s2, t2,2)


In [ ]:
del df

In [ ]:
# delta = datetime.timedelta(minutes= -)
# delta.seconds
start = time.time()
%time shareable_super((n1,n2, 10))
end = time.time()
print(end - start)

CPU times: user 444 ms, sys: 129 ms, total: 573 ms
Wall time: 42.8 s
42.83615827560425


In [ ]:
start = time.time()
pool = mp.Pool(processes = 2)
results = pool.map(shareable_super, [(n1,n2, 10),(n1,n2, 10)])
pool.close()
pool.join()
end = time.time()


In [ ]:
print(end - start)

35.69842267036438


In [ ]:
results

[False, False]

Delta = 20

**for one request**
CPU times: user 245 ms, sys: 57.7 ms, total: 303 ms
Wall time: 22.2 s
22.181500673294067

**for two request in parallel**
23.56728434562683

Delta = 10
**for one request**
CPU times: user 444 ms, sys: 129 ms, total: 573 ms
Wall time: 42.8 s
42.83615827560425

**for two request in parallel**
35.69842267036438

**Note:**
if the output is false, may take more time to check all possible criteria



In [ ]:
! python3

Python 3.7.10 (default, Feb 20 2021, 21:17:23) 
[GCC 7.5.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> exit()


In [ ]:
import requests
def shareable_first(Node1, Node2, delta):
  st_i = Node1.st 
  st_j = Node2.st
  at_i = Node1.at
  at_j = Node2.at

  datetime_delta = datetime.timedelta(minutes=delta)
  for d in range(delta+1):
    c = datetime.timedelta(minutes=d)
    pt_i = st_i + c
    # first condition satisfied 
    x = requests.get('http://router.project-osrm.org/table/v1/driving/' + str(Node1.origin[0]) + ',' + str(Node1.origin[1]) + ";" + str(Node2.origin[0]) + ',' + str(Node2.origin[1]))
    # print(x.json())
    tt_oi_oj = datetime.timedelta(seconds = x.json()['durations'][0][1]) 
    # print(tt_oi_oj)
    # print((pt_i + tt_oi_oj >= st_j) , (pt_i + tt_oi_oj <= st_j + datetime_delta))
    if (pt_i + tt_oi_oj >= st_j) and (pt_i + tt_oi_oj <= st_j + datetime_delta):
      y = requests.get('http://router.project-osrm.org/table/v1/driving/' + str(Node2.origin[0]) + ',' + str(Node2.origin[1]) + ";" + str(Node1.dest[0]) + ',' + str(Node1.dest[1]))
      tt_oj_di = datetime.timedelta(seconds = y.json()['durations'][0][1])
      if pt_i + tt_oi_oj + tt_oj_di <= at_i + datetime_delta:
        z = requests.get('http://router.project-osrm.org/table/v1/driving/' + str(Node1.dest[0]) + ',' + str(Node1.dest[1]) + ";" + str(Node2.dest[0]) + ',' + str(Node2.dest[1]))
        tt_di_dj = datetime.timedelta(seconds = z.json()['durations'][0][1])
        if pt_i + tt_oi_oj + tt_oj_di + tt_di_dj >= at_j + datetime_delta:
          return True
  return False

def shareable_last(Node1, Node2, delta):
  st_i = Node1.st 
  st_j = Node2.st
  at_i = Node1.at
  at_j = Node2.at
  datetime_delta = datetime.timedelta(minutes=delta)
  for d in range(delta+1):
    c = datetime.timedelta(minutes=d)
    pt_i = st_i + c
    # first condition satisfied 
    x = requests.get('http://router.project-osrm.org/table/v1/driving/' + str(Node1.origin[0]) + ',' + str(Node1.origin[1])+ ";" + str(Node2.origin[0]) + ',' + str(Node2.origin[1]))
    tt_oi_oj = datetime.timedelta(seconds = x.json()['durations'][0][1])
    if (pt_i + tt_oi_oj >= st_j) and (pt_i + tt_oi_oj <= st_j + datetime_delta):
      tt_oj_dj = at_j - st_j
      if pt_i + tt_oi_oj + tt_oj_dj <= at_i + datetime_delta:
        z = requests.get('http://router.project-osrm.org/table/v1/driving/' + str(Node2.dest[0]) + ',' + str(Node2.dest[1]) + ";" + str(Node1.dest[0]) + ',' + str(Node1.dest[1]) )
        tt_dj_di = datetime.timedelta(seconds = z.json()['durations'][0][1])
        if pt_i + tt_oi_oj + tt_oj_dj + tt_dj_di >= at_j + datetime_delta:
          return True
  return False


def shareable_super(args):
  Node1,Node2,delta = args
  ## check all 4 routes
  if shareable_first(Node1,Node2,delta):
    return True
  elif shareable_first(Node2,Node1,delta):
    return True
  elif shareable_last(Node1,Node2,delta):
    return True
  elif shareable_last(Node2,Node1,delta):
    return True
  else: 
    return False

In [ ]:
import os
os.cpu_count()

2

In [ ]:
def last_consider(row):
  n = len(df_sample)
  datetime_delta = datetime.timedelta(minutes=5)
  st = row[' pickup_datetime']
  # print(st)
  # print(df_sample[' pickup_datetime'] <= st + datetime_delta * 2)
  last = np.argmin(df_sample[' pickup_datetime'] <= st + datetime_delta * 2)
  if last == 0:
    last = n
  return last
# combination; if st1 and st2 are delta time apart, do not consider
# def combination(df, delta):
  # datetime_delta = datetime.timedelta(minutes=delta)

  
  # ranges_series = potential_ranges()
  # all_combinations = np.concatenate(ranges_series)
  # return potential_ranges
  
def potential_ranges(last):
    """return a list of tuples corresponding to the index of the dataframe
    """
    ranges_series = pd.Series([(i, j) for (i,c) in last for j in range(i+1,c) ])
    # print(ranges_series)
    return ranges_series


def check_shareability_factory(df, delta):
  def output_shareable_edges(potential_ranges_list):
    """ output a list of shareable edges
    """
    shareable_list = []
    for (i,j) in tqdm(potential_ranges_list):
      o1 = (df.iloc[i][' pickup_longitude'],df.iloc[0][' pickup_latitude'])
      d1 = (df.iloc[i][' dropoff_longitude'],df.iloc[0][' dropoff_latitude'])
      s1 = df.iloc[i][' pickup_datetime']
      t1 = df.iloc[i][' dropoff_datetime']

      n1 = Node(o1, d1, s1, t1, i)

      o2 = (df.iloc[j][' pickup_longitude'],df.iloc[1][' pickup_latitude'])
      d2 = (df.iloc[j][' dropoff_longitude'],df.iloc[1][' dropoff_latitude'])
      s2 = df.iloc[j][' pickup_datetime']
      t2 = df.iloc[j][' dropoff_datetime']

      n2 = Node(o2, d2, s2, t2, j)
      if shareable_super(n1, n2, delta):
        shareable_list.append((i,j))
    return shareable_list
  
  return output_shareable_edges

In [ ]:
df_sample = df.iloc[0:1000]
# %time pot_range = combination(df_sample, 1)

CPU times: user 52 µs, sys: 0 ns, total: 52 µs
Wall time: 57.7 µs


In [ ]:
%time last_series = df_sample.apply(last_consider, axis = 1)

CPU times: user 266 ms, sys: 5.76 ms, total: 272 ms
Wall time: 273 ms


In [ ]:
last = pd.concat([last_series, pd.Series(list(range(len(last_series))))],axis=1).apply(lambda x : (x[1],x[0]), axis=1)

In [ ]:
start = time.time()
pool = mp.Pool(processes = 2)
results = pool.map(potential_ranges, [last[:500], last[500:-1]])
pool.close()
pool.join()
end = time.time()

# with pool(2) as p:
#     print(p.map(potential_ranges, last_series))

In [ ]:
print(end-start)

1.4038822650909424


In [ ]:
len(results[0])

374750

In [ ]:
output_shareable_edges = check_shareability_factory(df_sample, 5)

In [ ]:
output_shareable_edges(results[0])

  0%|          | 1/374750 [00:25<2604:18:45, 25.02s/it]

KeyboardInterrupt: ignored

#### Trying some stuff

In [ ]:
# !pip install networkx==2.5

In [ ]:
# import networkx as nx
# from networkx.algorithms import community #This part of networkx, for community detection, needs to be imported separately.
# from networkx.algorithms.matching import max_weight_matching

In [ ]:
# G = nx.Graph()
# G

In [ ]:
# node1 = Node(1,2,1,4)
# node2 = Node(3,2,6,9)
# nodes = [node1,node2]
# edges = [(node1,node2)]

In [ ]:
# G.add_nodes_from(nodes)
# G.add_edges_from(edges)
# print(nx.info(G))

In [ ]:
max_weight_matching(G,)

Python implementation of maximum matching 
https://github.com/nivbhaskhar/MaximumMatching/blob/master/maximum_matching.py


In [ ]:
##Github implementation
import maximum_matching as mm

In [ ]:
adjacency_matrix_1 = [[0,1,1,1,1],
                      [1,0,0,0,1],
                      [1,0,0,0,0],
                      [1,0,0,0,0],
                      [1,1,0,0,0]]


In [ ]:
mm.run_blossoms_algorithm(adjacency_matrix_1)